In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.animation as animation
from IPython.display import HTML
import seaborn as sb

vgsales = pd.read_csv("vgsales.csv")
vgsales = pd.DataFrame(vgsales)

vgsales['Year'] = vgsales['Year'].fillna(-1)
vgsales['Year'] = vgsales['Year'].astype(int)
vgsales.set_index('Rank', inplace=True, drop=True)
vgsales

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...
16596,Woody Woodpecker in Crazy Castle 5,GBA,2002,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16597,Men in Black II: Alien Escape,GC,2003,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16598,SCORE International Baja 1000: The Official Game,PS2,2008,Racing,Activision,0.00,0.00,0.00,0.00,0.01


# Objetivo do Trabalho

Meu objetivo nesse trabalho será avaliar a quantidade de vendas cumulativas de cada gênero específico de jogos ao longo dos anos. Para visualizar isso de uma forma prática e interessante, criarei uma chamada "bar chart race", um tipo de animação onde cada frame representa um ano específico, com o gráfico de barras representando a quantidade as vendas de cada gênero. Dessa forma, será possível analizar de forma mais direta os dados, a fim de descobrir se existe algum tipo de tendência no mercado de jogos em relação os gêneros mais populares com o passar do tempo. 

## Manipulação dos Dados

Entretanto, para montar uma visualização como essa, é primeiro preciso tratar a base de dados original. Como se pode ver na tabela a seguir, a quantidade de vendas retratada na tabela não são as vendas cumulativas até aquele ano, e sim as vendas específicas que ocorreram naquele ano. Além disso, há ainda diversos jogos cujo ano de lançamento não está disponível na base. Esses jogos serão descartados na análise que eu farei posteriormente.

In [2]:
vgsales[(vgsales["Year"] != -1)].groupby(["Year"],as_index=False)[["Global_Sales"]].sum().head()

,Year,Global_Sales
0,1980,11.38
1,1981,35.77
2,1982,28.86
3,1983,16.79
4,1984,50.36


Agora, farei uma série de operações para criar um novo dataframe, que contém as vendas cumulativas de cada gênero por ano. Para isso, criarei um subset do dataframe original através do comando .groupby, agrupando os dados pelo ano e gênero de cada jogo, e somando o valor de "Global_Sales" de cada combinação. Além disso, já definirei algumas variáveis que serão utilizadas posteriormente no processo.

In [3]:
vgsales_year = vgsales[(vgsales["Year"] != -1)].groupby(["Year", "Genre"],as_index=False)[["Global_Sales"]].sum()

genres = set()
years = []
cumulative = []
genreslist = []

vgsales_year.head()

,Year,Genre,Global_Sales
0,1980,Action,0.34
1,1980,Fighting,0.77
2,1980,Misc,2.71
3,1980,Shooter,7.07
4,1980,Sports,0.49


Com o subset e as variáveis auxiliares prontas, podemos começar as operações. Primeiramente, criarei um conjunto de todos os gêneros diferentes (a variável "genres"), e utilizarei dois conjuntos de for loops para "escanear" cada linha da tabela vgsales_year, e atribuir os valores de venda corretamente para cada ano e gênero distinto. Por último, criarei um dataframe com base nesses dados, que será utilizado para a animação seguinte.

In [4]:
for tupla in vgsales_year.itertuples():
    genres.add(tupla.Genre)


for genre in genres:
    sales = 0
    for tupla in vgsales_year.itertuples():
        if tupla.Genre == genre:
            years.append(tupla.Year)
            sales += tupla.Global_Sales
            cumulative.append(sales)
            genreslist.append(tupla.Genre)

dic = {"Year": years,
       "Cumulative_Sales": cumulative,
       "Genre": genreslist}

df = pd.DataFrame(dic)

df.head(35)

,Year,Cumulative_Sales,Genre
0,1980,0.49,Sports
1,1981,1.28,Sports
2,1982,2.33,Sports
3,1983,5.53,Sports
4,1984,11.71,Sports
5,1985,13.67,Sports
6,1986,19.24,Sports
7,1987,22.96,Sports
8,1988,26.56,Sports
9,1989,32.28,Sports


Como pode ser visto na tabela acima, a lógica aparenta ter dado certo! Agora, está na hora de montar a função que criará o gráfico. Algo a se observar é que o comportamento padrão do argumento colors de um gráfico de barras atribui as cores se baseando na "posição" de cada barra (por exemplo, a primeira barra é sempre azul, a segunda é sempre vermelha). Como nosso objetivo é criar uma animação com base no tempo, faz mais sentido que as cores sejam atribuídas ao *gênero* que a barra representa, para que a comparação entre anos diferentes seja muito mais intuitiva. Portanto, criaremos uma palheta de cores genérica para atribuir a cada gênero específico.

In [5]:
hex = ['#D2D8B3', '#C38D94', '#C3D350', '#A1E8AF',
     '#3C91E6', '#FE5F55', '#90DDF0', '#808F85',
     '#F4B942', '#6A994E', '#747C92', '#E086D3']
colors = dict(zip(genres, hex))
print(colors)

{'Sports': '#D2D8B3', 'Simulation': '#C38D94', 'Racing': '#C3D350', 'Shooter': '#A1E8AF', 'Misc': '#3C91E6', 'Action': '#FE5F55', 'Fighting': '#90DDF0', 'Strategy': '#808F85', 'Platform': '#F4B942', 'Puzzle': '#6A994E', 'Adventure': '#747C92', 'Role-Playing': '#E086D3'}


In [6]:
fig, ax = plt.subplots(figsize = (15, 9))
plt.close()
def draw_barchart(year):
        dff = (df[df["Year"].eq(year)].sort_values(by="Cumulative_Sales", ascending=True))
        ax.clear()

        ax.barh(dff["Genre"], dff["Cumulative_Sales"], color = [colors[x] for x in dff["Genre"]])
        dx = dff["Cumulative_Sales"].max() / 200

        for i, (sales, genre) in enumerate(zip(dff["Cumulative_Sales"], dff["Genre"])):
                ax.text(sales-dx, i-.07, genre, size=13, weight=600, ha="right", va="bottom")
                ax.text(sales-dx, i,  "  " + str(round(sales)), size=13, ha="left", va="center")

        ax.text(1, 0.4, year, transform=ax.transAxes, color='#777777', size=46, ha="right", weight=800)
        ax.text(0, 1.06, "Vendas de jogos (em milhares)", transform=ax.transAxes, size=12, color="#777777", ha="left", va="center")

        ax.xaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
        ax.xaxis.set_ticks_position("top")
        ax.tick_params(axis="x", colors="#333333", labelsize=12)
        ax.set_yticks([])
        ax.margins(0, 0.01)
        ax.grid(which="major", axis="x", linestyle="-")
        ax.set_axisbelow(True)

        ax.text(0, 1.12, "Venda total acumulada de jogos por gênero, de 1980 a 2015", transform=ax.transAxes, size=24, weight=600, ha="left", va="top")
        plt.box(False)


In [7]:
animator = animation.FuncAnimation(fig, draw_barchart, frames=range(1980, 2016))
animation = HTML(animator.to_jshtml())
plt.close()

In [8]:
animation